In [1]:
import pandas as pd
import numpy as np
import scipy.spatial
from numba import njit

In [2]:
df = pd.read_csv("train.csv")[-100000:]
df = df[::-1].reset_index().drop(["index"], axis=1)
df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1632182400,11,48.0,232.695000,232.800000,232.240000,232.275000,1.035123e+02,232.569697,NaN
1,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN
2,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN
3,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN
4,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN
...,...,...,...,...,...,...,...,...,...,...
99995,1631753640,11,59.0,266.454000,266.650000,266.260000,266.338000,3.623673e+01,266.436999,0.005342
99996,1631753640,12,249.0,0.336325,0.336415,0.335783,0.336100,2.338795e+05,0.336187,0.001845
99997,1631753640,13,485.0,0.117135,0.117222,0.116986,0.117134,1.663125e+06,0.117131,-0.000118
99998,1631753640,10,441.0,3024.661517,3046.000000,3017.000000,3038.457000,1.133077e+02,3034.417218,0.021506


In [3]:
df = df[["timestamp", "Asset_ID", "VWAP"]]
df_main = df.copy()
df_main["timestamp"] = pd.to_datetime(df_main["timestamp"], unit="s", infer_datetime_format=True)

In [4]:
def getDtVWAP(df, aID):
    interim = df.loc[df.Asset_ID == aID].drop(["Asset_ID"], axis=1)
    dt = interim["timestamp"].to_numpy()
    interim["VWAP_adj"] = interim["VWAP"] / interim["VWAP"].rolling(10).mean()
    vwap = interim["VWAP_adj"].to_numpy()
    return dt, vwap

def compute_cost_matrix(X, Y, metric="euclidean"):
    X, Y = np.atleast_2d(X, Y)
    return scipy.spatial.distance.cdist(X.T, Y.T, metric=metric)

@njit
def compute_accumulated_cost_matrix_subsequence_dtw(C):
    N, M = C.shape
    D = np.zeros((N, M))
    D[:, 0] = np.cumsum(C[:, 0])
    D[0, :] = C[0, :]
    for n in range(1, N):
        for m in range(1, M):
            D[n, m] = C[n, m] + min(D[n-1, m], D[n, m-1], D[n-1, m-1])
    return D

@njit
def compute_optimal_warping_path_subsequence_dtw(D, m=-1):
    N, M = D.shape
    n = N - 1
    if m < 0:
        m = D[N - 1, :].argmin()
    P = [(n, m)]

    while n > 0:
        if m == 0:
            cell = (n - 1, 0)
        else:
            val = min(D[n-1, m-1], D[n-1, m], D[n, m-1])
            if val == D[n-1, m-1]:
                cell = (n-1, m-1)
            elif val == D[n-1, m]:
                cell = (n-1, m)
            else:
                cell = (n, m-1)
        P.append(cell)
        n, m = cell
    P.reverse()
    P = np.array(P)
    return P

@njit
def minima_from_matching_function(delta, rho, tau, num=None):
    delta_tmp = delta.copy()
    pos = []
    num_pos = 0
    rho = int(rho)
    if num is None:
        num = len(delta)
    while num_pos < num and np.sum(delta_tmp < tau) > 0:
        pos.append(np.argmin(delta_tmp))
        num_pos += 1
        delta_tmp[max(0, np.argmin(delta_tmp) - rho):min(np.argmin(delta_tmp) + rho, len(delta))] = np.inf
    return np.array(pos).astype(np.int64)

@njit
def matches_dtw(pos, D):
    matches = np.zeros((len(pos), 2)).astype(np.int64)
    for k in range(len(pos)):
        matches[k, 1] = pos[k]
        matches[k, 0] = compute_optimal_warping_path_subsequence_dtw(D, m=pos[k])[0, 1]
    return matches

def return_matches(X, Y):
    C = compute_cost_matrix(X, Y, metric="euclidean")
    D = compute_accumulated_cost_matrix_subsequence_dtw(C)
    pos = minima_from_matching_function(D[-1, :] / C.shape[0], C.shape[0] // 2, 0.001, None)
    return matches_dtw(pos, D)

def batch_dtw(start_idx, end_idx, vwap, lb, dt):
    batch_res = {}
    for i in range(start_idx, end_idx):
        X, Y = vwap[i:i + lb], vwap[i + lb + 1:]
        batch_res[dt[i]] = return_matches(X, Y)
    return batch_res

def forecast_returns(ret, matches, forecast_window):
    tails, forecast = [], []
    for i in range(len(matches)):
        tails.append(matches[i][1])
    for j in tails:
        forecast.append(ret[j + forecast_window])
    return np.mean(forecast)

def forecast_batch_returns(vwap, matches, forecast_window):
    forecast = {}
    for k, v in matches.items():
        forecast[k] = forecast_returns(vwap, v, forecast_window)
    return forecast

In [5]:
for i in range(14):
    dt, vwap = getDtVWAP(df_main, i)
    matches = batch_dtw(0, len(vwap), vwap, 10, dt)
    batch_forecast = forecast_batch_returns(vwap, matches, 10)
    pd.Series(list(batch_forecast.values()), index=batch_forecast.keys()).to_csv("forecast_" + str(i) + ".csv")

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
